In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np

from src.utils.labels import Labels
from src.scrapers.ams_bgt_scraper import scrape_amsterdam_bgt, parse_polygons
from src.utils.las_utils import read_las, label_and_save_las, get_bbox_from_las_file
from src.region_growing.label_connected_comp import LabelConnectedComp

In [ ]:
# Select the file to process.
in_file = '../datasets/pointcloud/grown_2397_9705.las'
out_file = '../datasets/pointcloud/cars_2397_9705.las'
ahn_data_file = '../datasets/ahn/ahn_2397_9705.npz'

In [ ]:
pointcloud = read_las(in_file)
las_labels = pointcloud.label
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T

In [ ]:
from src.utils.ahn_utils import load_ahn_tile
ahn_tile = load_ahn_tile(ahn_data_file)

In [ ]:
exclude_labels = (Labels.GROUND, Labels.BUILDING)

In [ ]:
bbox = get_bbox_from_las_file(in_file)

In [ ]:
bgt_road_polygons = []
for layer in ["BGT_WGL_rijbaan_lokale_weg", "BGT_WGL_parkeervlak"]:
    json_content = scrape_amsterdam_bgt(layer, bbox=bbox)
    road_polygons, _ = parse_polygons(json_content, offset_meter=-0.3)
    bgt_road_polygons += road_polygons

In [ ]:
# TODO dit verwerken in de pipeline?
lcc = LabelConnectedComp(Labels.CAR, exclude_labels, octree_level=10,
                         min_component_size=100, max_above_ground=3,
                         min_area_thresh=6, max_area_thresh=16)
lcc._set_mask(las_labels)
lcc._convert_input_cloud(points)
lcc._label_connected_comp()
las_labels = lcc._fill_car_like_components(bgt_road_polygons, ahn_tile)

In [ ]:
label_and_save_las(pointcloud, las_labels, out_file)